In [1]:
from google.colab import drive
drive.mount('/content/data') # 드라이브 연결

Mounted at /content/data


In [2]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [ ]:
training_data = datasets.FashionMNIST(
    root="desktop",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="desktop",
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
train_dataloader = DataLoader(training_data,batch_size=64)
test_dataloader = DataLoader(test_data,batch_size=64)

In [5]:
class NeuralNetwork(nn.Module): # 설계도
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
# 모델 선언
model = NeuralNetwork() # 생성

In [9]:
# loss 선언
loss_fn = nn.CrossEntropyLoss()

In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
## backpropagation(역전파) 순서  optimizer.zero_grad() -> loss.backwodr() -> optimizer.stet()

In [13]:
def train_loop(dataloader, model, loss_fn, optimizer): # 전체데이터 학습(1 apochs)
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 배치 사이즈 만큼 꺼내서 넘겨줌(60000개를 64씩 끊어서 학습 시켜줌)
        # Compute prediction and loss
        pred = model(X)          # 64개 이미지(X) -> forward
        loss = loss_fn(pred, y)  # loss 계산 
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test_loop(dataloader, model, loss_fn): # 학스비 아니니까 optimizer는 안들어감 
    size = len(dataloader.dataset) 
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:  # 전체데이터가 돌떄까지 for문
            pred = model(X)      # 64개 이미지(X) 
            test_loss += loss_fn(pred, y).item() # 파이썬 결과값으로(item) 바꿔주고 합침
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() #pred.argmax(맞췄으면) 1반환하고 sum 해줘라 -> 맞춘 개수
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(correct):>0.1f}%, Avg loss: {test_loss:>8f} \n") # appoch에대한 accuaracy 랑 loss 평균

In [ ]:
# hyper parameter 선언
learning_rate = 1e-3
epochs = 10 # 전체 데이터 5 번 학습 

for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

## saving Models

In [ ]:
torch.save(model.state_dict(), "data/model.pth")
a
print("Saved PyTorch Model State to model.pth")

## loading models

In [15]:
model = NeuralNetwork()
model.load_state_dict(torch.load("data/model.pth"))

FileNotFoundError: ignored

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()  # 모델을 예측 모드로 변경 
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')